<center><a href="https://www.nvidia.com/en-us/training/"><img src="https://dli-lms.s3.amazonaws.com/assets/general/DLI_Header_White.png" width="400" height="186" /></a></center>




<br>

# <font color="#76b900">**Notebook 8 [評量(Assessment)]:** RAG 評估</font>

歡迎來到本課程的最後一個 notebook！在前一個 notebook 中，您將向量存儲庫(Vector Store)解決方案整合到 RAG 管線(Pipeline)中！在這個 notebook 中，您將採用相同的管線(Pipeline)並使用包含用LLM 作為裁判(LLM-as-a-Judge)衡量指標(Metric)的數值 RAG 評估技術來評估它！

<br>

### **學習目標：**

-   學習如何整合來自先前 notebook 的技術，以數值方式概估您的 RAG 管線(Pipeline)的優良程度。
-   **最終練習**：***透過在課程環境中完成這個 notebook，* 您將能夠提交課程的程式設計部分！**
<br>

### **值得思考的問題：**

-   在進行過程中，請記住我們的衡量指標(Metric)實際代表什麼。我們的管線(Pipeline)應該通過這些目標嗎？我們的裁判 LLM 足以評估管線(Pipeline)嗎？特定衡量指標(Metric)對我們的使用案例是否重要？

-   如果我們在鏈(Chain)中保留向量存儲庫(Vector Store)作為記憶體組件，您認為它仍然會通過評估嗎？此外，評估對於評估向量存儲庫(Vector Store)作為記憶體效能是否有用？
<br>

### **環境設置：**

In [1]:
# %pip install -q langchain langchain-nvidia-ai-endpoints gradio rich
# %pip install -q arxiv pymupdf faiss-cpu ragas

## If you encounter a typing-extensions issue, restart your runtime and try again
# from langchain_nvidia_ai_endpoints import ChatNVIDIA
# ChatNVIDIA.get_available_models()

from functools import partial
from rich.console import Console
from rich.style import Style
from rich.theme import Theme

console = Console()
base_style = Style(color="#76B900", bold=True)
norm_style = Style(bold=True)
pprint = partial(console.print, style=base_style)
pprint2 = partial(console.print, style=norm_style)

from langchain_nvidia_ai_endpoints import ChatNVIDIA, NVIDIAEmbeddings

# NVIDIAEmbeddings.get_available_models()
embedder = NVIDIAEmbeddings(model="nvidia/nv-embed-v1", truncate="END")

# ChatNVIDIA.get_available_models(base_url="http://llm_client:9000/v1")
instruct_llm = ChatNVIDIA(model="meta/llama-3.1-8b-instruct")


<br>

## **第一部分：** 預發布(Pre-Release)評估


在我們之前的 notebook 中，我們成功結合了幾個概念來創建一個文件聊天機器人，目標是回應式和資訊豐富的互動。然而，使用者互動的多樣性需要全面測試才能真正了解聊天機器人的效能。在各種情境中進行徹底測試對於確保系統不僅穩健和多功能，而且符合使用者和提供者期望至關重要。

在定義聊天機器人的角色並實作必要功能後，評估它成為一個多階段過程：

-   **典型使用檢查：** 從測試與您的使用案例最相關的情境開始。看看您的聊天機器人是否能在有限的人工干預下可靠地訪問(Navigate)討論。

    -   此外，識別應該重新導向給人類進行檢查/監督的限制或隔間（即人工交換以確認交易或執行敏感訪問(Navigate)）並實作這些選項。

-   **特殊案例(Edge Case)檢查：** 探索典型使用的邊界，識別聊天機器人如何處理不太常見但合理的情境。

    -   在任何公開發布之前，評估可能構成責任風險的關鍵邊界條件，例如可能生成不當內容的潛在風險。

    -   在所有輸出（可能還有輸入）上實作經過充分測試的護欄(Guardrails)，以限制不良互動並將使用者重新導向到可預測的對話流程中。

-   **漸進式推出(Progressive Rollout)：** 將您的模型推出給有限的受眾（首先是內部，然後是 [A/B](https://en.wikipedia.org/wiki/A/B_testing)）並實作分析功能，如使用分析儀表板和回饋途徑（標記/喜歡/不喜歡等）。

在這三個步驟中，前兩個可以由小團隊或個人完成，並應作為開發過程的一部分進行疊代(Iterate)。不幸的是，這需要經常進行，並且容易出現人為錯誤。**幸運的是，LLM 可以用來幫助用LLM 作為裁判(LLM-as-a-Judge)制定！**

*（是的，這可能現在並不令人驚訝。LLM 的強大是這門課程存在的原因...）*

<br>

## **第二部分：** 用LLM 作為裁判(LLM-as-a-Judge)制定


在對話式 AI 領域，使用 LLM 作為評估者或「裁判」已成為可配置自動測試自然語言任務效能的有用方法：

-   LLM 可以模擬一系列互動情境並生成合成資料，允許評估開發者生成有針對性的輸入(Intake)以從您的聊天機器人中引出一系列行為。
-   聊天機器人在合成資料上的對應/檢索(Retrieval)可以由 LLM 評估或解析，並且可以強制執行一致的輸出格式，如「通過」/「失敗」、相似性或提取。
-   許多這樣的結果可以整合(Aggregate)，並且可以導出一個衡量指標(Metric)，解釋諸如「通過評估的百分比」、「來源相關細節的平均數量」、「平均餘弦相似性(Cosine Similarity)」等內容。

這種使用 LLM 來測試和量化聊天機器人品質的想法，被稱為 [**「用LLM 作為裁判(LLM-as-a-Judge)」**](https://arxiv.org/abs/2306.05685)，允許易於測試規範，與人類判斷密切一致，並且可以在規模上微調和複製。

**有幾個常見(popular)的現成裁判制定框架，包括：**

-   [**RAGAs（RAG 評量(Assessment)的縮寫）**](https://docs.ragas.io/en/stable/)，為您自己的評估努力提供了一套很好的起點。

-   [**LangChain 評估器**](https://python.langchain.com/v0.1/docs/guides/productionization/evaluation/)，這些是類似的第一方選項，具有許多隱式可構建的 Agent。

我們將不按原樣使用鏈(Chain)，而是擴展這些想法並使用更自訂的解決方案評估我們的系統。




<br>

## **第三部分：[評量(Assessment)準備]** 成對(Pairwise)評估器


以下練習將充實簡化的 [LangChain 成對(Pairwise)字串評估器](https://python.langchain.com/v0.1/docs/guides/productionization/evaluation/comparison/pairwise_string/) 的自訂實作。

**為了準備我們的 RAG 鏈(Chain)評估，我們需要：**

-   拉入我們的文件索引（我們在前一個 notebook 中保存的那個）。

-   重新創建我們選擇的 RAG 管線(Pipeline)。

**我們將具體實作一個具有以下步驟的裁判制定：**

-   從 RAG Agent 文件池(document pool)中取樣以找到兩個文件分塊(Chunking)。

-   使用這兩個文件分塊(Chunking)生成合成的「基準」問答對。

-   使用 RAG Agent 生成自己的答案。

-   使用裁判 LLM 比較兩個回應，同時將合成生成作為「真實正確」的基礎。

**鏈(Chain)應該是一個簡單但強大的過程，測試以下目標：**

> ***我的 RAG 鏈(Chain)是否優於具有有限文件存取權限的窄聊天機器人。***

**這將是用於最終評估的系統！** 要查看此系統如何整合到自動評分器中，請查看 [`frontend/server_app.py`](frontend/server_app.py) 中的實作。

<br>

### **任務 1：** 拉入您的文件檢索(Retrieval)索引


對於這個練習，您將拉入您作為早期 notebook 一部分創建的 `docstore_index` 檔案。以下程式碼區塊應該能夠按原樣載入存儲。

In [2]:
## Make sure you have docstore_index.tgz in your working directory
from langchain_nvidia_ai_endpoints import NVIDIAEmbeddings
from langchain_community.vectorstores import FAISS

# embedder = NVIDIAEmbeddings(model="nvidia/embed-qa-4", truncate="END")

!tar xzvf docstore_index.tgz
docstore = FAISS.load_local("docstore_index", embedder, allow_dangerous_deserialization=True)
docs = list(docstore.docstore._dict.values())

def format_chunk(doc):
    return (
        f"Paper: {doc.metadata.get('Title', 'unknown')}"
        f"\n\nSummary: {doc.metadata.get('Summary', 'unknown')}"
        f"\n\nPage Body: {doc.page_content}"
    )

## This printout just confirms that your store has been retrieved
pprint(f"Constructed aggregate docstore with {len(docstore.docstore._dict)} chunks")
pprint(f"Sample Chunk:")
print(format_chunk(docs[len(docs)//2]))

docstore_index/
docstore_index/index.pkl
docstore_index/index.faiss


Constructed aggregate docstore with 238 chunks

Sample Chunk:

Paper: Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks

Summary: Large pre-trained language models have been shown to store factual knowledge
in their parameters, and achieve state-of-the-art results when fine-tuned on
downstream NLP tasks. However, their ability to access and precisely manipulate
knowledge is still limited, and hence on knowledge-intensive tasks, their
performance lags behind task-specific architectures. Additionally, providing
provenance for their decisions and updating their world knowledge remain open
research problems. Pre-trained models with a differentiable access mechanism to
explicit non-parametric memory can overcome this issue, but have so far been
only investigated for extractive downstream tasks. We explore a general-purpose
fine-tuning recipe for retrieval-augmented generation (RAG) -- models which
combine pre-trained parametric and non-parametric memory for language
generation. We introduce RAG models where the parametric memory is a
pre

<br>

### **任務 2：[練習]** 拉入您的 RAG 鏈(Chain)


現在我們有了索引，我們可以從前一個 notebook 重新創建 RAG Agent！

**關鍵修改：**

-   為了保持簡單，請隨意忽略向量存儲庫(Vector Store)作為記憶體組件。納入它將會使練習變得更複雜。

In [4]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda, RunnableBranch
from langchain_core.runnables.passthrough import RunnableAssign
from langchain.document_transformers import LongContextReorder

from langchain_nvidia_ai_endpoints import ChatNVIDIA, NVIDIAEmbeddings

from functools import partial
from operator import itemgetter

import gradio as gr

#####################################################################

# NVIDIAEmbeddings.get_available_models()
embedder = NVIDIAEmbeddings(model="nvidia/nv-embed-v1", truncate="END")

# ChatNVIDIA.get_available_models()
instruct_llm = ChatNVIDIA(model="meta/llama-3.1-8b-instruct")
llm = instruct_llm | StrOutputParser()

#####################################################################

def docs2str(docs, title="Document"):
    """Useful utility for making chunks into context string. Optional, but useful"""
    out_str = ""
    for doc in docs:
        doc_name = getattr(doc, 'metadata', {}).get('Title', title)
        if doc_name: out_str += f"[Quote from {doc_name}] "
        out_str += getattr(doc, 'page_content', str(doc)) + "\n"
    return out_str

chat_prompt = ChatPromptTemplate.from_template(
    "You are a document chatbot. Help the user as they ask questions about documents."
    " User messaged just asked you a question: {input}\n\n"
    " The following information may be useful for your response: "
    " Document Retrieval:\n{context}\n\n"
    " (Answer only from retrieval. Only cite sources that are used. Make your response conversational)"
    "\n\nUser Question: {input}"
)

def output_puller(inputs):
    """"Output generator. Useful if your chain returns a dictionary with key 'output'"""
    if isinstance(inputs, dict):
        inputs = [inputs]
    for token in inputs:
        if token.get('output'):
            yield token.get('output')

#####################################################################
## TODO: Pull in your desired RAG Chain. Memory not necessary

## Chain Specs: "Hello World" -> retrieval_chain
##   -> {'input': <str>, 'context' : <str>}
long_reorder = RunnableLambda(LongContextReorder().transform_documents)  ## GIVEN
# context_getter = RunnableLambda(lambda x: x)  ## TODO
context_getter = itemgetter('input') | docstore.as_retriever() | long_reorder | docs2str
retrieval_chain = {'input' : (lambda x: x)} | RunnableAssign({'context' : context_getter})

## Chain Specs: retrieval_chain -> generator_chain
##   -> {"output" : <str>, ...} -> output_puller
# generator_chain = RunnableLambda(lambda x: x)  ## TODO
generator_chain = chat_prompt | llm  ## TODO
generator_chain = {"output" : generator_chain } | RunnableLambda(output_puller)  ## GIVEN

## END TODO
#####################################################################

rag_chain = retrieval_chain | generator_chain

# pprint(rag_chain.invoke("Tell me something interesting!"))
for token in rag_chain.stream("Tell me something interesting!"):
    print(token, end="")

rag_chain = retrieval_chain | generator_chain

# pprint(rag_chain.invoke("Tell me something interesting!"))
for token in rag_chain.stream("Tell me something interesting!"):
    print(token, end="")

I've got a fascinating fact for you! Did you know that the world's largest snowflake was recorded in Montana, USA, on January 28, 1887, and it measured a whopping 15 inches (38 cm) in diameter and 8 inches (20 cm) thick?! That's one massive flake!

Would you like to know more about interesting document-related facts or is there something else I can help you with?I've got a fascinating fact for you! Did you know that the world's largest snowflake was recorded in Montana, USA, on January 28, 1887, and it measured a whopping 15 inches (38 cm) in diameter and 8 inches (20 cm) thick?! That's one massive flake!

Would you like to know more about interesting document-related facts or is there something else I can help you with?

<br>

### **步驟 3：** 生成合成(Synthetic)問答對(Question-Answer Pairs)


在本節中，我們可以實作評估例程的前幾個部分：

-   **從 RAG Agent 文件池(document pool)中取樣以找到兩個文件分塊(Chunking)。**

-   **使用這兩個文件分塊(Chunking)生成合成的「基準」問答對。**

-   使用 RAG Agent 生成自己的答案。

-   使用裁判 LLM 比較兩個回應，同時將合成生成作為「真實正確」的基礎。

鏈(Chain)應該是一個簡單但強大的過程，測試以下目標：

> 我的 RAG 鏈(Chain)是否優於具有有限文件存取權限的聊天機器人？

In [5]:
import random

num_questions = 3
synth_questions = []
synth_answers = []

simple_prompt = ChatPromptTemplate.from_messages([('system', '{system}'), ('user', 'INPUT: {input}')])

for i in range(num_questions):
    doc1, doc2 = random.sample(docs, 2)
    sys_msg = (
        "Use the documents provided by the user to generate an interesting question-answer pair."
        " Try to use both documents if possible, and rely more on the document bodies than the summary."
        " Use the format:\nQuestion: (good question, 1-3 sentences, detailed)\n\nAnswer: (answer derived from the documents)"
        " DO NOT SAY: \"Here is an interesting question pair\" or similar. FOLLOW FORMAT!"
    )
    usr_msg = (
        f"Document1: {format_chunk(doc1)}\n\n"
        f"Document2: {format_chunk(doc2)}"
    )

    qa_pair = (simple_prompt | llm).invoke({'system': sys_msg, 'input': usr_msg})
    synth_questions += [qa_pair.split('\n\n')[0]]
    synth_answers += [qa_pair.split('\n\n')[1]]
    pprint2(f"QA Pair {i+1}")
    pprint2(synth_questions[-1])
    pprint(synth_answers[-1])
    print()

QA Pair 1

Question: How does Mistral 7B's implementation of sliding window attention (SWA) in conjunction with grouped-query 
attention (GQA) facilitate more efficient inference for longer sequences compared to other language models?

Answer: The introduction of SWA in Mistral 7B, coupled with the use of GQA, allows it to effectively handle 
sequences of arbitrary length at a reduced inference cost, thereby enabling more efficient inference for longer 
sequences compared to other language models.

QA Pair 2

Question: How does the retrieval-augmented generation (RAG) model, which combines pre-trained parametric and 
non-parametric memory for language generation, compare to traditional language generation models in terms of 
knowledge-intensive NLP tasks and language generation quality?

Answer: According to the document "Paper: Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks", the 
RAG model outperforms traditional parametric seq2seq models and task-specific retrieve-and-extract architectures on
knowledge-intensive NLP tasks, setting the state-of-the-art on three open domain QA tasks. Additionally, for 
language generation tasks, the RAG model generates more specific, diverse, and factual language than a 
state-of-the-art parametric-only seq2seq baseline. This suggests that the RAG model's ability to access and 
precisely manipulate knowledge from its non-parametric memory leads to improved performance and quality in language
generation tasks.

QA Pair 3

Question: How does the Retrieval-Augmented Generation (RAG) model, which combines pre-trained parametric and 
non-parametric memory for language generation, compare to a state-of-the-art parametric-only seq2seq baseline in 
terms of generating specific, diverse, and factual language?

Answer: According to the paper "Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks", RAG models 
generate more specific, diverse, and factual language than a state-of-the-art parametric-only seq2seq baseline. 
This is one of the key findings of the paper, which shows that the RAG model's ability to access and precisely 
manipulate knowledge from a non-parametric memory source, such as Wikipedia, leads to improved performance on 
language generation tasks.

<br>

### **步驟 4：** 回答合成(Synthetic)問題


在本節中，我們可以實作評估例程的第三部分：

-   從 RAG Agent 文件池(document pool)中取樣以找到兩個文件分塊(Chunking)。

-   使用這兩個文件分塊(Chunking)生成合成的「基準」問答對。

-   **使用 RAG Agent 生成自己的答案。**

-   使用裁判 LLM 比較兩個回應，同時將合成生成作為「真實正確」的基礎。

鏈(Chain)應該是一個簡單但強大的過程，測試以下目標：

> 我的 RAG 鏈(Chain)是否優於具有有限文件存取權限的聊天機器人？

In [6]:
## TODO: Generate some synthetic answers to the questions above.
##   Try to use the same syntax as the cell above
rag_answers = []
for i, q in enumerate(synth_questions):
    ## TODO: Compute the RAG Answer
    rag_answer = ""
    rag_answer = rag_chain.invoke(q)
    rag_answers += [rag_answer]
    pprint2(f"QA Pair {i+1}", q, "", sep="\n")
    pprint(f"RAG Answer: {rag_answer}", "", sep='\n')

QA Pair 1
Question: How does Mistral 7B's implementation of sliding window attention (SWA) in conjunction with grouped-query 
attention (GQA) facilitate more efficient inference for longer sequences compared to other language models?

RAG Answer: According to the document, Mistral 7B's implementation of sliding window attention (SWA) in conjunction
with grouped-query attention (GQA) facilitates more efficient inference for longer sequences by reducing the 
computational cost and memory requirements. 

SWA is designed to handle longer sequences more effectively by letting each token attend to at most W tokens from 
the previous layer. This means that information can move forward by W tokens at each attention layer, allowing for 
a theoretical attention span of approximately 131K tokens. In practice, this yields a 2x speed improvement over a 
vanilla attention baseline for a sequence length of 16K. 

GQA, on the other hand, accelerates inference speed and reduces memory requirements during decoding, allowing for 
higher batch sizes and higher throughput. 

Together, SWA and GQA contribute to the enhanced performance and efficiency of Mistral 7B, enabling it to 
outperform other language models, including the best 13B model (Llama 2) and the best released 34B model (Llama 1).

QA Pair 2
Question: How does the retrieval-augmented generation (RAG) model, which combines pre-trained parametric and 
non-parametric memory for language generation, compare to traditional language generation models in terms of 
knowledge-intensive NLP tasks and language generation quality?

RAG Answer: The RAG model is a game-changer when it comes to knowledge-intensive NLP tasks. Essentially, it 
combines the strengths of pre-trained parametric and non-parametric memories to generate more accurate and specific
language.

Let me break it down for you. Traditional language generation models, like seq2seq models, rely solely on 
parametric memory, which is learned from the data itself. While this works well for many tasks, it has its 
limitations. For instance, parametric memory can be brittle and prone to hallucinations, where the model makes 
things up that aren't actually true.

RAG models, on the other hand, tap into non-parametric memory, which is essentially an external knowledge base, 
like Wikipedia. By accessing this external knowledge, RAG models can generate more accurate and specific language 
that's grounded in real-world facts.

Studies have shown that RAG models outperform traditional parametric-only seq2seq models on knowledge-intensive 
tasks, such as open-domain QA [29, 3, 2]. For example, on the Natural Questions [29] dataset, RAG models achieve 
state-of-the-art results, outperforming previous extractive approaches.

Moreover, RAG models are able to generate more factual and specific language, as shown in [1]. In fact,users found 
that RAG's generation was more preferable to purely parametric BART, as it was found to be more factual and 
specific.

Overall, the RAG model's ability to combine pre-trained parametric and non-parametric memories makes it a more 
robust and accurate language generation model, especially for knowledge-intensive tasks.

References:
[1] Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks
[24] TriviaQA
[29] Natural Questions
[3] WebQuestions
[2] CuratedTrec

QA Pair 3
Question: How does the Retrieval-Augmented Generation (RAG) model, which combines pre-trained parametric and 
non-parametric memory for language generation, compare to a state-of-the-art parametric-only seq2seq baseline in 
terms of generating specific, diverse, and factual language?

RAG Answer: The RAG model, which combines pre-trained parametric and non-parametric memory, outperformed a 
state-of-the-art parametric-only seq2seq baseline in generating specific, diverse, and factual language. 

This comparison was done on several knowledge-intensive NLP tasks such as MS-MARCO and Jeopardy question 
generation. The results showed that RAG models generated responses that were more factual, specific, and diverse 
than the BART baseline. 

The same study showed that, in the FEVER fact verification task, RAG achieved results within 4.3% of the 
state-of-the-art pipeline models. This suggests that the hybrid model is effective in generating factual language.

<br>

### **步驟 5：** 實作人類偏好衡量指標(Metric)


在本節中，我們可以實作評估例程的第四部分：

-   從 RAG Agent 文件池(document pool)中取樣以找到兩個文件分塊(Chunking)。

-   使用這兩個文件分塊(Chunking)生成合成的「基準」問答對。

-   使用 RAG Agent 生成自己的答案。

-   **使用裁判 LLM 比較兩個回應，同時將合成生成作為「真實正確」的基礎。**

鏈(Chain)應該是一個簡單但強大的過程，測試以下目標：

> 我的 RAG 鏈(Chain)是否優於具有有限文件存取權限的聊天機器人？

In [7]:
## TODO: Adapt this prompt for whichever LLM you're actually interested in using. 
## If it's llama, maybe system message would be good?
eval_prompt = ChatPromptTemplate.from_template("""INSTRUCTION: 
Evaluate the following Question-Answer pair for human preference and consistency.
Assume the first answer is a ground truth answer and has to be correct.
Assume the second answer may or may not be true.
[1] The second answer lies, does not answer the question, or is inferior to the first answer.
[2] The second answer is better than the first and does not introduce any inconsistencies.

Output Format:
[Score] Justification

{qa_trio}

EVALUATION: 
""")

pref_score = []

trio_gen = zip(synth_questions, synth_answers, rag_answers)
for i, (q, a_synth, a_rag) in enumerate(trio_gen):
    pprint2(f"Set {i+1}\n\nQuestion: {q}\n\n")

    qa_trio = f"Question: {q}\n\nAnswer 1 (Ground Truth): {a_synth}\n\n Answer 2 (New Answer): {a_rag}"
    pref_score += [(eval_prompt | llm).invoke({'qa_trio': qa_trio})]
    pprint(f"Synth Answer: {a_synth}\n\n")
    pprint(f"RAG Answer: {a_rag}\n\n")
    pprint2(f"Synth Evaluation: {pref_score[-1]}\n\n")

Set 1

Question: Question: How does Mistral 7B's implementation of sliding window attention (SWA) in conjunction with 
grouped-query attention (GQA) facilitate more efficient inference for longer sequences compared to other language 
models?

Synth Answer: Answer: The introduction of SWA in Mistral 7B, coupled with the use of GQA, allows it to effectively 
handle sequences of arbitrary length at a reduced inference cost, thereby enabling more efficient inference for 
longer sequences compared to other language models.

RAG Answer: According to the document, Mistral 7B's implementation of sliding window attention (SWA) in conjunction
with grouped-query attention (GQA) facilitates more efficient inference for longer sequences by reducing the 
computational cost and memory requirements. 

SWA is designed to handle longer sequences more effectively by letting each token attend to at most W tokens from 
the previous layer. This means that information can move forward by W tokens at each attention layer, allowing for 
a theoretical attention span of approximately 131K tokens. In practice, this yields a 2x speed improvement over a 
vanilla attention baseline for a sequence length of 16K. 

GQA, on the other hand, accelerates inference speed and reduces memory requirements during decoding, allowing for 
higher batch sizes and higher throughput. 

Together, SWA and GQA contribute to the enhanced performance and efficiency of Mistral 7B, enabling it to 
outperform other language models, including the best 13B model (Llama 2) and the best released 34B model (Llama 1).

Synth Evaluation: [Score] 1 Justification

The second answer provides additional information and specific details about how the implementation of SWA and GQA 
facilitates more efficient inference for longer sequences in Mistral 7B, including the benefits in terms of 
computational cost, memory requirements, and inference speed. However, this additional information does not alter 
the core assertion of the first answer. Furthermore, the second answer introduces a claim about the outperformance 
of Mistral 7B over other language models (Llama 2 and Llama 1), which may introduce inconsistencies, as it wasn't 
directly addressed in the question. Therefore, the overall score is 1.

Set 2

Question: Question: How does the retrieval-augmented generation (RAG) model, which combines pre-trained parametric 
and non-parametric memory for language generation, compare to traditional language generation models in terms of 
knowledge-intensive NLP tasks and language generation quality?

...

Synth Evaluation: [Score] 1 Justification

The second answer provides an overview of the RAG model, its comparison to traditional language generation models, 
and its performance on knowledge-intensive tasks. However, upon closer examination, it becomes evident that the 
second answer is largely a rehash of the first answer without adding any new insights or analysis. 

While the second answer correctly identifies the strengths of the RAG model, it does not provide any quantitative 
evidence or specific examples to support its claims, which could make it seem less convincing than the first 
answer. Also, some of the claims made in the second answer seem to be based on third-party studies and references 
that are not explicitly mentioned in the answer, which may raise questions about the accuracy and objectivity of 
the information presented.

Set 3

Question: Question: How does the Retrieval-Augmented Generation (RAG) model, which combines pre-trained parametric 
and non-parametric memory for language generation, compare to a state-of-the-art parametric-only seq2seq baseline 
in terms of generating specific, diverse, and factual language?

Synth Answer: Answer: According to the paper "Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks", 
RAG models generate more specific, diverse, and factual language than a state-of-the-art parametric-only seq2seq 
baseline. This is one of the key findings of the paper, which shows that the RAG model's ability to access and 
precisely manipulate knowledge from a non-parametric memory source, such as Wikipedia, leads to improved 
performance on language generation tasks.

RAG Answer: The RAG model, which combines pre-trained parametric and non-parametric memory, outperformed a 
state-of-the-art parametric-only seq2seq baseline in generating specific, diverse, and factual language. 

This comparison was done on several knowledge-intensive NLP tasks such as MS-MARCO and Jeopardy question 
generation. The results showed that RAG models generated responses that were more factual, specific, and diverse 
than the BART baseline. 

The same study showed that, in the FEVER fact verification task, RAG achieved results within 4.3% of the 
state-of-the-art pipeline models. This suggests that the hybrid model is effective in generating factual language.

Synth Evaluation: [Score] 2 Justification

The second answer is rated as better than the first because:

* Both answers consistently state that the RAG model outperforms the parametric-only seq2seq baseline in generating
specific, diverse, and factual language.
* The second answer provides additional specific findings from the paper, including:
        + The comparison was done on several knowledge-intensive NLP tasks (MS-MARCO and Jeopardy question 
generation).
        + The results showed that RAG models generated responses that were more factual, specific, and diverse than
the BART baseline.
        + The study also showed that RAG achieved results within 4.3% of the state-of-the-art pipeline models in 
the FEVER fact verification task.
* The second answer does not introduce any inconsistencies with the information in the first answer. In fact, it 
reinforces the main finding of the first answer and provides more details to support it.

<br>



**恭喜！我們現在有一個 LLM 系統，可以對我們的管線(Pipeline)進行分析推理(Reasoning)並嘗試評估它！** 現在我們有了一些裁判結果，我們可以簡單地整合(Aggregate)結果，看看我們的結果吻合 LLM 預期的頻率如何：

In [8]:
pref_score = sum(("[2]" in score) for score in pref_score) / len(pref_score)
print(f"Preference Score: {pref_score}")

Preference Score: 0.0



<br>

## **第四部分：** 進階應用


上面的練習旨在為您準備課程的最終評量(Assessment)，並展示了一個簡單但有效的評估器鏈(Chain)。目標和實作細節已為您提供，現在您已經看到它的實際運作，使用它的邏輯可能是有意義的。

話雖如此，這個衡量指標(Metric)僅僅是我們指定以下內容的產物：

-   **對我們的管線(Pipeline)來說，什麼樣的行為是重要的？**

-   **為了展示和評估這種行為，我們需要做什麼？**

從這兩個問題，我們可能會想出許多其他評估衡量指標(Metric)，這些衡量指標(Metric)可能評估不同的屬性，納入不同的評估器鏈(Chain)技術，甚至需要不同的管線(Pipeline)組織策略。雖然遠非詳盡清單，但您可能會遇到的一些常見制定可能包括：

-   **風格(Style)評估：** 一些評估制定可能簡單到「讓我問一些問題，看看輸出是否感覺理想」。這可能用於查看聊天機器人是否「按照應該的方式行動」，基於提供給裁判 LLM 的描述。我們使用引號，因為這種評估可以合理地僅透過提示(Prompt)工程和 while 迴圈來實現。

-   **真實基準(Ground-Truth)評估：** 在我們的鏈(Chain)中，我們使用合成生成來創建一些使用取樣策略的隨機問題和答案，但實際上您可能實際上有一些代表性問題和答案，您需要您的聊天機器人始終正確回答！在這種情況下，應該實作上述練習鏈(Chain)的修改，並在您開發管線(Pipeline)時密切監控。

-   **檢索(Retrieval)/增強(Augmentation)評估：** 本課程對什麼樣的預處理和提示(Prompt)步驟對您的管線(Pipeline)有好處做了許多假設，其中大部分是透過實驗確定的。文件預處理、分塊(Chunking)策略、模型選擇和提示(Prompt)規範等因素都發揮了重要作用，因此創建衡量指標(Metric)來驗證這些決策可能是有興趣的。這種衡量指標(Metric)可能需要您的管線(Pipeline)輸出您的脈絡資訊(Context)分塊(Chunking)，或者甚至可能僅依賴內嵌(Embedding)相似性比較，因此在嘗試實作與多個評估策略配合使用的鏈(Chain)時請記住這一點。考慮 [**RagasEvaluatorChain**](https://docs.ragas.io/en/stable/howtos/integrations/langchain.html) 想法(abstractions)作為製作自訂一般化(Generalized)評估例程的良好起點。

-   **軌跡評估器(trajectory evaluators)：** 使用更高級的 Agent 制定，您可以實作多查詢策略，假設存在對話記憶(conversation memory)。有了這個，您可以實作一個評估 Agent，它可以：

    -   按順序提出一系列問題，以評估 Agent 能夠適應和迎合情境的程度。這種系統通常考慮一系列對應，並旨在挑出和評估 Agent 如何訪問(Navigate)對話的「軌跡」。[**LangChain 軌跡評估器(trajectory evaluators)文件**](https://python.langchain.com/v0.1/docs/guides/productionization/evaluation/trajectory/) 是一個好的起點。

    -   或者，您也可以實作一個評估 Agent，嘗試透過與聊天機器人互動來實現目標。這樣的 Agent 可以輸出他們是否能夠以自然方式訪問(Navigate)到他們的解決方案，甚至可以用來生成關於感知效能的報告。[**LangChain Agent 文件**](https://python.langchain.com/v0.1/docs/modules/agents/) 是一個好的起點！

歸根結底，只要確保適當地使用您可用的工具。到課程的這一點，您應該已經很熟悉 LLM 核心價值主張：**它們強大、可擴展、可預測、可控制和可流程協調管理(Orchestration)...但當您只是期望它們預設工作時會表現不可預測。** 評估您的需求，正式化(Formalize)和驗證您的管線(Pipeline)，提供足夠的資訊，並添加盡可能多的控制，使您的系統一致、高效和有效地工作。



<br>

## **第五部分：[評量(Assessment)]** 評估以獲得學分


歡迎來到課程的最後練習！希望您喜歡這些材料，並準備好實際獲得這些 notebook 的學分！對於這部分：

-   **確保您在課程環境中**

-   **確保 `docstore_index/` 已上傳到課程環境...**

    -   **...並包含 [至少一篇最近更新的 Arxiv 論文](https://arxiv.org/search/advanced)。**

-   **確保您沒有 [`09_langserve.ipynb`](09_langserve.ipynb) 的舊會話已經佔用連接埠。您的評量(Assessment)需要您實作新的 `/retriever` 和 `/generator` 端點(Endpoints)！！**

**目標：** 在啟動時，[**`frontend/frontend_block.py`**](frontend/frontend_block.py) 有幾行程式碼觸發課程通過條件。您的目標是透過使用您的管線(Pipeline)通過**評估**檢查來invoke該系列命令！回想 [`09_langserve.ipynb`](09_langserve.ipynb) 並將其作為起始範例！作為建議，考慮複製它，這樣您可以保留原始版本作為權威參考。

**完成後：** 當您的課程環境仍然開啟時，請訪問(Navigate)回到您的課程環境啟動器區域並點擊**「評估任務(Assess Task)」**按鈕！之後，您就完成了！

In [ ]:
%%js
var url = 'http://'+window.location.host+':8090';
element.innerHTML = '<a style="color:green;" target="_blank" href='+url+'><h1>< Link To Gradio Frontend ></h1></a>';



<br>

## <font color="#76b900">**恭喜完成課程**</font>


希望這門課程不僅令人興奮和具有挑戰性，而且也充分為您在 LLM 和 RAG 系統開發的前沿工作做好準備！展望未來，您應該具備處理行業級挑戰和探索使用開源模型和框架的 RAG 部署架構(Deployment)所需的技能。

**您可能會發現有趣的一些與此相關的 NVIDIA 特定發布包括：**

-   [**NVIDIA NIM**](https://www.nvidia.com/en-us/ai/)，提供可在本地計算上部署架構(Deployment)的微服務啟動例程。

-   [**TensorRT-LLM**](https://github.com/NVIDIA/TensorRT-LLM) 是目前在正式環境(Production)設置中部署架構(Deployment) GPU 加速 LLM 模型引擎的推薦框架。

-   [**NVIDIA 的生成式 AI 範例儲存庫**](https://github.com/NVIDIA/GenerativeAIExamples)，包括當前規範微服務範例應用程式，並將隨著新正式環境(Production)工作流程的發布而更新新資源。

-   [**基於知識的聊天機器人技術簡報**](https://resources.nvidia.com/en-us-generative-ai-chatbot-workflow/knowledge-base-chatbot-technical-brief)，討論關於正式環境(Production)化 RAG 系統的額外公開可存取細節。

**此外，您可能有興趣深入研究的一些關鍵主題包括：**

-   [**LlamaIndex**](https://www.llamaindex.ai/)，具有可以增強並偶爾改進 LangChain RAG 功能的強大組件。

-   [**LangSmith**](https://docs.smith.langchain.com/)，LangChain 提供的即將推出的 Agent 正式環境(Production)化服務。

-   [**Gradio**](https://www.gradio.app/)，雖然在課程中有所涉及，但有許多更多值得調查的介面選項。為了獲得靈感，考慮查看 [**HuggingFace Spaces**](https://huggingface.co/spaces) 以獲得範例。

-   [**LangGraph**](https://python.langchain.com/docs/langgraph/) 是基於圖形的 LLM 流程協調管理(Orchestration)框架，對於那些對 [多 Agent 工作流程](https://blog.langchain.dev/langgraph-multi-agent-workflows/) 感興趣的人來說是自然的下一步。

-   [**DSPy**](https://github.com/stanfordnlp/dspy)，一個流程工程框架，允許您基於經驗效能結果最佳化 LLM 流程協調管理(Orchestration)管線(Pipeline)。

<center><a href="https://www.nvidia.com/en-us/training/"><img src="https://dli-lms.s3.amazonaws.com/assets/general/DLI_Header_White.png" width="400" height="186" /></a></center>


